In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max.columns', None)

In [31]:
df = pd.read_csv(r"C:\Users\Admin\OneDrive\Pictures\dylan profile\dylan gr11\Dylan\Data analyst Dylan\PANDAS\SANAYAN LANG BOSS\Supermarket\SuperMarket Analysis.csv")

In [48]:
# DATA IS CLEAN. BUT WE ARE CREATING A BOXPLOT FIRST TO LOCATE IF THERE ARE OUTLIERS IN THE DATA. 

df[['Sales', 'cogs', 'Tax 5%', 'Rating', 'Quantity', 'Unit price']].boxplot()
plt.show()

# VALIDATING THE OUTLIERS
df.sort_values(by='Sales', ascending=False).head(10)
df.sort_values(by='cogs', ascending=False).head(10)
df.sort_values(by='Tax 5%', ascending=False).head(10)
# OUTLIERS ARE VALID

<Axes: >

In [49]:
# 1.1 GETTING THE SUM OF SALES OF ALL BRANCHES

df.groupby('Branch')['Sales'].sum()

In [50]:
# 1.2 GETTING THE SUM OF SALES OF ALL PRODUCT LINES

df.groupby('Product line')['Sales'].sum()

In [51]:
# 1.3 EXTRACTING THE DAY OF THE WEEK AND THE HOUR OF THE DAY FROM THE DATE & TIME COLUMN
from datetime import datetime

df['Date'] = pd.to_datetime(df['Date'])
df['Day_of_week'] = df['Date'].dt.day_name()
df['Time_of_day'] = pd.to_datetime(df['Time'], format='%I:%M:%S %p').dt.time
df['Time_of_day'] = pd.to_datetime(df['Time_of_day'], format= '%H:%M:%S')
df['Date_time'] = df['Date'] + pd.to_timedelta(df['Time_of_day'].dt.hour, unit='h') \
                                            + pd.to_timedelta(df['Time_of_day'].dt.minute, unit='m') \
                                            + pd.to_timedelta(df['Time_of_day'].dt.second, unit='s')
df.drop(columns=['Time_of_day'], inplace=True)
df['Hour_of_day'] = df['Date_time'].dt.hour

df.groupby('Hour_of_day')['Sales'].sum()
df.groupby('Day_of_week')['Sales'].sum()

# WILL USE POWER QUERY TO EXTRACT THE DAY OF THE WEEK AND THE HOUR OF THE DAY

In [52]:
# 1.4 GETTING THE AVERAGE TRANSACTION VALUE

df['Sales'].mean()

In [37]:
# 1.5 ADDING A Transaction_category COLUMN

cogs_conditions = [df['Sales'] >= 471.350250,
                  (df['Sales'] >= 253.848000) & (df['Sales'] < 471.350250),
                  df['Sales'] < 253.848000]

cogs_categories = ['High Value', 'Middle Value', 'Low Value']

df['Transaction_category'] = np.select(cogs_conditions, cogs_categories)

# WILL USE POWER QUERY TO ADD THE SAME COLUMN IN THE DATASET

In [39]:
# 2.1 GETTING THE RATIO OF EACH GENDER'S PURCHASE BY PRODCT LINE

subtotals_gender_by_product_line = df.groupby(['Gender', 'Product line']).size().reset_index(name='count')
totals_gender_by_product_line = subtotals_gender_by_product_line.groupby('Gender')['count'].transform('sum')
subtotals_gender_by_product_line['ratio'] = subtotals_gender_by_product_line['count']/totals_gender_by_product_line
ratio_gender_product_line = subtotals_gender_by_product_line.pivot(index='Product line', columns='Gender', values='ratio')

# CREATING A FILE PATH TO EXPORT THE DATAFRAMES TO

filepath = r"C:\Users\Admin\OneDrive\Pictures\dylan profile\dylan gr11\Dylan\Data analyst Dylan\PANDAS\SANAYAN LANG BOSS\Supermarket\supermarket_ratios.xlsx"

with pd.ExcelWriter(filepath, engine='openpyxl', mode='w') as writer:
    ratio_gender_product_line.to_excel(writer, sheet_name='gender_product')

# CREATING AN EXCEL EXPORTER FUNCTION:

def excel_exporter(df, file_path, sheet_name):
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name)

In [53]:
# 2.2 GETTING THE CORRELATION BETWEEN QUANTITY AND RATING

df['Quantity'].corr(df['Rating'])

In [41]:
# 2.3 GETTING THE RATIO OF EACH CUSTOMER TYPE'S PURCHASE BY PAYMENT TYPE

subtotals_customertype_by_payment = df.groupby(['Customer type', 'Payment']).size().reset_index(name='count')
totals_customertype_by_payment = subtotals_customertype_by_payment.groupby('Customer type')['count'].transform('sum')
subtotals_customertype_by_payment['ratio'] = subtotals_customertype_by_payment['count']/totals_customertype_by_payment
ratio_customertype_by_payment = subtotals_customertype_by_payment.pivot(index='Payment', columns='Customer type', values='ratio')

# CALLING EXCEL EXCEL EXPORTER
excel_exporter(ratio_customertype_by_payment, filepath, 'customertype_payment')

In [42]:
# 2.4 GETTING THE RATIO OF EACH CUSTOMER TYPE'S TRANSACTION CATEGORY

subtotals_customertype_by_transaction = df.groupby(['Customer type', 'Transaction_category']).size().reset_index(name='count')
totals_customertype_by_transaction = subtotals_customertype_by_transaction.groupby('Customer type')['count'].transform('sum')
subtotals_customertype_by_transaction['ratio'] = subtotals_customertype_by_transaction['count']/totals_customertype_by_transaction
ratio_customertype_by_transaction = subtotals_customertype_by_transaction.pivot(index='Transaction_category', columns='Customer type', values='ratio')

# CALLING EXCEL EXCEL EXPORTER
excel_exporter(ratio_customertype_by_transaction, filepath, 'customertype_transaction')

In [54]:
# 2.5 GETTING THE SUM OF SALE OF EACH CUSTOMER TYPES

df.groupby('Customer type')['Sales'].sum()

In [55]:
# 2.6 GETTING THE AVERAGE RATINGS OF EACH CUSTOMER TYPES

df.groupby('Customer type')['Rating'].mean()

In [44]:
# 3.1 GETTING THE GROSS INCOME PERCENTAGE OF EACH BRANCH

sum_of_sales_per_branch = df.groupby('Branch')['Sales'].sum().reset_index(name='sum_of_sales')
sum_of_cogs_per_branch = df.groupby('Branch')['cogs'].sum().reset_index(name='sum_of_cogs')
branch_data = pd.merge(sum_of_sales_per_branch, sum_of_cogs_per_branch, on='Branch')
branch_data['gross_income'] = branch_data['sum_of_sales'] - branch_data['sum_of_cogs']
branch_data['gross_income_margin'] = (branch_data['gross_income']/branch_data['sum_of_sales'])*100
branch_data

# SO THAT"S WHY GROSS MARGIN PERCENTAGE IS CONSISTENT ACROSS THE DATAFRAME... I WAS ONLY ABLE TO UNDERSTAND IT NOW.
# CALLING EXCEL EXCEL EXPORTER
excel_exporter(branch_data, filepath, 'branch_data')

In [56]:
# 3.2 CHECKING THE GROSS INCOME PER PRODUCT LINE

df.groupby('Product line')['gross income'].sum()

In [57]:
# 3.3 (1.2) GETTING THE SUM OF SALES OF ALL PRODUCT LINES

df.groupby('Product line')['Sales'].sum()

In [58]:
# ASKING CHATGPT TO CREATE A BOXPLOT LIKE THE ONE AT THE START OF THE PROJECT BUT CUSTOMIZED ACCORDING TO THE THEME OF THE DASHBOARD.
    
box_color = "#2ECC71"      # green/teal for box outlines
median_color = "#F1C40F"   # gold/yellow for median
bg_color = "none"          # transparent
grid_color = "#555555"     # subtle gray grid
font_family = "Segoe UI"   # matches Power BI default

cols = ['Sales', 'cogs', 'Tax 5%', 'Rating', 'Quantity', 'Unit price']

fig, ax = plt.subplots(figsize=(12, 6))

# Transparent background for seamless Power BI integration
fig.patch.set_alpha(0)
ax.set_facecolor("none")

# Boxplot with custom props
boxprops = dict(color=box_color, linewidth=1.5)
whiskerprops = dict(color=box_color, linewidth=1.5)
capprops = dict(color=box_color, linewidth=1.5)
medianprops = dict(color=median_color, linewidth=2)
flierprops = dict(marker='o', markerfacecolor=median_color, markersize=6, linestyle='none', alpha=0.7)

df[cols].boxplot(ax=ax, boxprops=boxprops, whiskerprops=whiskerprops,
                 capprops=capprops, medianprops=medianprops, flierprops=flierprops)

# Labels and title in white
#ax.set_title("Sales, COGS, Tax, Rating, Quantity, Unit Price", fontsize=14, fontname=font_family, fontweight='bold', color="white")
ax.set_ylabel("Value", fontsize=20, fontname=font_family, color="white")
ax.tick_params(axis='x', labelrotation=30, colors="white")
ax.tick_params(axis='y', colors="white")
plt.xticks(fontsize=20)

# Subtle grid
ax.grid(color=grid_color, linestyle='--', linewidth=0.5, alpha=0.5)

plt.tight_layout()
plt.show()

In [47]:
### QUESTIONS

### 1. Sales Performance Analysis

# 1.1 Which branch generates the highest revenue?

# 1.2 What is the sum of sales per product line?

# 1.3 Which day of the week or time of day has the highest sales?

# 1.4 What is the average transaction value?

# 1.5 Which payment method is most popular among high-value transactions?



### 2. Customer Behavior & Segmentation

# 2.1 Do men and women prefer different product lines?

# 2.2 Is there a correlation between the quantity purchased and the customer rating?

# 2.3 - 2.6 Are there noticeable differences in purchasing behavior between Members and Normal customers?
    # 2.3 Payment Type
    # 2.4 Transaction Type
    # 2.5 Sales
    # 2.6 Ratings



### 3. Profitability & Operational Insights

# 3.1 Which branch or city has the highest gross income margin?

# 3.2 What is the gross income per product line?

# 3.3 Are there underperforming product lines in ratings?



### 4. Marketing & Business Strategy

# Which product line should the supermarket promote more based on profitability and popularity?

# Which branch should the company invest more in based on sales growth potential?

# Can we segment customers based on their buying behavior or product preferences?

# Are promotions or discounts during certain times/days likely to improve revenue?

# Can we identify cross-sell or upsell opportunities based on purchase patterns?